In [1]:
# !pip install torch torchvision torchaudio
# !pip install matplotlib
# !pip install Pillow

# Libraries

In [1]:
import torch
import matplotlib.pyplot as plt
from PIL import Image

print(torch.__version__)
print(torch.cuda.is_available())

2.5.1+cpu
False


In [2]:
import json
import cv2
import numpy as np
import os

# Isolation


In [18]:
img_path = r"C:\Users\shawn\Downloads\SMART\Radiographs\Radiographs\797.JPG"
image = Image.open(img_path)
image.show()

In [19]:
from PIL import Image, ImageDraw

# img_path = r"C:\Users\shawn\Downloads\SMART\Segmentation\Segmentation\teeth_mask\797.JPG"
img_path = r"C:\Users\shawn\Downloads\SMART\Radiographs\Radiographs\997.JPG"
json_path = r"C:\Users\shawn\Downloads\SMART\Segmentation\Segmentation\teeth_bbox.json"

image = Image.open(img_path)
with open(json_path, 'r') as f:
    bbox_data = json.load(f)

# Get bounding boxes for .JPG
sample_entry = next(item for item in bbox_data if item["External ID"].lower() == "997.jpg")
boxes = sample_entry["Label"]["objects"]

# Draw
draw = ImageDraw.Draw(image)
for obj in boxes:
    box = obj["bounding box"]
    draw.rectangle(box, outline="red", width=2)
    draw.text((box[0], box[1]-10), obj["title"], fill="yellow")

# Show the image with boxes
image.show()


In [23]:
#IDEA? Boxes seem to be placed on the bottom left but are all bunched up together, so lets try using data augmentation to make the image a certain way. 
#note: img 748 has no teeth LOL so no boxes but this works for diff images.
# Rotating Image
img_path = r"C:\Users\shawn\Downloads\SMART\Radiographs\Radiographs\670.JPG"
image = Image.open(img_path).rotate(90, expand=True)
rotated_width, rotated_height = image.size
print(f"Rotated image size: {rotated_width} x {rotated_height}")

with open(json_path, 'r') as f:
    bbox_data = json.load(f)

entry = next(item for item in bbox_data if item["External ID"].lower() == "670.jpg")
boxes = entry["Label"]["objects"]

draw = ImageDraw.Draw(image)

for obj in boxes:
    x1, y1, x2, y2 = obj["bounding box"]
    draw.rectangle([x1, y1, x2, y2], outline="red", width=2)
    draw.text((x1, y1 - 10), obj["title"], fill="yellow")

image.show()



Rotated image size: 840 x 1615
